In [1]:
from dotenv import load_dotenv
import os
import asyncio
_ = load_dotenv(".ollama.env") 

In [16]:
from ollama import AsyncClient

client = AsyncClient(
    host="https://ollama.com",
    headers={'Authorization': 'Bearer ' + os.environ.get('OLLAMA_API_KEY')}
)

In [17]:
models = ["gpt-oss:20b", "ministral-3:14b"] # "qwen3-vl:235b-cloud", "deepseek-v3.2", 

In [18]:
from mercury import Columns, Chat, Message, ChatInput

In [31]:

left, right = Columns(2, border=True)

ColumnsBox(children=(ColumnOutput(layout=Layout(border_bottom='1px solid lightgray', border_left='1px solid li…

In [32]:

with left:
    left_chat = Chat()
with right:
    right_chat = Chat()

In [33]:
# list of messages
left_messages = []
right_messages = []


In [34]:
async def ollama_responder(model, messages):
    # chat(...) is a coroutine – we must await it
    stream = await client.chat(model=model, messages=messages, stream=True)
    # now 'stream' is an async iterator of parts
    async for part in stream:
        yield part["message"]["content"]
        

        
async def stream_responder(model, messages, bot_msg):
    print("model", model)
    async for chunk in ollama_responder(model, messages):
        bot_msg.append_markdown(chunk)

In [35]:
async def handle_submit(msg):

    # User prompt
    user_msg = Message(role="user", emoji="👤")
    user_msg.set_message(markdown=msg)
    left_chat.add(user_msg)
    right_chat.add(user_msg)

    # Left AI response shell
    left_bot_msg = Message(role="assistant", emoji="🤖")
    left_bot_msg.set_gradient_text("Thinking hard 🤔")
    left_chat.add(left_bot_msg)

    # Right AI response shell
    right_bot_msg = Message(role="assistant", emoji="🤖")
    right_bot_msg.set_bouncing_text("Thinking hard 🤔")
    right_chat.add(right_bot_msg)

    messages = [
      {
        'role': 'user',
        'content': 'Why is the sky blue? max 2 sentences',
      },
    ]
    # Start both streams in parallel
    await asyncio.gather(
        stream_responder(models[0], messages, left_bot_msg),
        stream_responder(models[1], messages, right_bot_msg),
    )


In [36]:
prompt = ChatInput()

In [37]:
if prompt.value:
    await handle_submit(prompt.value)
    

model gpt-oss:20b
model ministral-3:14b
